# seedbleniding

exp40,exp49 のseed42,614


In [2]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

import pickle

In [5]:
train_42 = pd.read_csv("../output/exp42_2_logistic_lagdiff_c3/oof_exp42_3_logistic_0.782316702096232_5fold_seed42.csv")
test_42 = pd.read_csv("../output/exp42_2_logistic_lagdiff_c3/test_exp42_3_logistic_0.782316702096232_5fold_seed42.csv")

train_616 = pd.read_csv("../output/exp42_3_616_logistic/oof_exp42_3_logistic_0.7824760851833245_5fold_seed616.csv")
test_616 = pd.read_csv("../output/exp42_3_616_logistic/test_exp42_3_logistic_0.7824760851833245_5fold_seed616.csv")



In [4]:
train_42.head()

customer_ID  target  exp42_3_logistic_oof
0  0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...       1              0.633278
1  00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...       0              0.040596
2  0004860c260168fcaad0734a1dfedb7ceb1a83aaac54e2...       1              0.999608
3  0004b8596c4946866d1b27a8b40488ecf49c6eae9c7bf4...       0              0.004494
4  000548e99fa24cef8377e68e602e4bd70d30500a007999...       0              0.001249

In [6]:
train_616.head()

customer_ID  target  exp42_3_logistic_oof
0  000041bdba6ecadd89a52d11886e8eaaec9325906c9723...       0              0.011146
1  000084e5023181993c2e1b665ac88dbb1ce9ef621ec537...       0              0.010803
2  0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...       1              0.663020
3  00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...       0              0.038112
4  0004e00358fc4dd633e491c8090a0794b6165ed9a7e60c...       0              0.001055

In [7]:
oof = train_42.merge(train_616,on = "customer_ID")
sub = test_42.merge(test_616,on = "customer_ID")


In [8]:
sub.head()

customer_ID  prediction_x  prediction_y
0  00000469ba478561f23a92a868bd366de6f6527a684c9a...      0.026778      0.026887
1  00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...      0.002919      0.002905
2  0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...      0.054579      0.053841
3  00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...      0.214095      0.210052
4  00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...      0.857592      0.856282

In [9]:
oof.head()

customer_ID  target_x  exp42_3_logistic_oof_x  target_y  exp42_3_logistic_oof_y
0  0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...         1                0.633278         1                0.663020
1  00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...         0                0.040596         0                0.038112
2  0004860c260168fcaad0734a1dfedb7ceb1a83aaac54e2...         1                0.999608         1                0.999507
3  0004b8596c4946866d1b27a8b40488ecf49c6eae9c7bf4...         0                0.004494         0                0.004395
4  000548e99fa24cef8377e68e602e4bd70d30500a007999...         0                0.001249         0                0.001304

In [10]:
oof["exp42_42+616_oof"] = (oof["exp42_3_logistic_oof_x"] + oof["exp42_3_logistic_oof_y"])/2
sub["prediction"] = (sub["prediction_x"] + sub["prediction_y"])/2 

In [12]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)


score = amex_metric(oof["target_x"], oof["exp42_42+616_oof"])
print("CV : ",score)

CV :  0.7828114243647253


In [13]:
oof[["customer_ID","exp42_42+616_oof"]].to_csv(f"exp42_oof_logist_{score}_seedblend.csv",index = False)
sub[["customer_ID","prediction"]].to_csv(f"exp51_sub_logist_{score}_seedblend.csv",index = False)


In [15]:

amex_metric(oof["target_x"], oof["exp42_3_logistic_oof_x"])

0.782316702096232